#Esercizio 2: Air quality

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [19]:
#lavoro sul dataset
df = pd.read_csv('AirQualityUCI.csv', sep=';')

df['CO(GT)'] = pd.to_numeric(df['CO(GT)'].str.replace(',', '.'))
df = df[['Date', 'Time', 'CO(GT)']].replace(-200, np.nan).dropna()


In [20]:
# calcolo media e assegno etichetta
daily_mean = df.groupby('Date')['CO(GT)'].transform('mean')
df['Label'] = np.where(df['CO(GT)'] > daily_mean, "scarsa qualità dell'aria", "buona qualità dell'aria")
print(df.head(24))

          Date      Time  CO(GT)                     Label
0   10/03/2004  18.00.00     2.6  scarsa qualità dell'aria
1   10/03/2004  19.00.00     2.0  scarsa qualità dell'aria
2   10/03/2004  20.00.00     2.2  scarsa qualità dell'aria
3   10/03/2004  21.00.00     2.2  scarsa qualità dell'aria
4   10/03/2004  22.00.00     1.6   buona qualità dell'aria
5   10/03/2004  23.00.00     1.2   buona qualità dell'aria
6   11/03/2004  00.00.00     1.2   buona qualità dell'aria
7   11/03/2004  01.00.00     1.0   buona qualità dell'aria
8   11/03/2004  02.00.00     0.9   buona qualità dell'aria
9   11/03/2004  03.00.00     0.6   buona qualità dell'aria
11  11/03/2004  05.00.00     0.7   buona qualità dell'aria
12  11/03/2004  06.00.00     0.7   buona qualità dell'aria
13  11/03/2004  07.00.00     1.1   buona qualità dell'aria
14  11/03/2004  08.00.00     2.0   buona qualità dell'aria
15  11/03/2004  09.00.00     2.2   buona qualità dell'aria
16  11/03/2004  10.00.00     1.7   buona qualità dell'ar

In [24]:
# 3 valori più alti al giorno
higher = df.groupby('Date').apply(lambda g: g.nlargest(3, 'CO(GT)'))
print(higher.head(10))

                       Date      Time  CO(GT)                     Label  Week
Date                                                                         
01/01/2005 7128  01/01/2005  18.00.00     4.7  scarsa qualità dell'aria    53
           7129  01/01/2005  19.00.00     3.6  scarsa qualità dell'aria    53
           7127  01/01/2005  17.00.00     3.0  scarsa qualità dell'aria    53
01/02/2005 7863  01/02/2005  09.00.00     4.8  scarsa qualità dell'aria     5
           7865  01/02/2005  11.00.00     4.5  scarsa qualità dell'aria     5
           7866  01/02/2005  12.00.00     4.5  scarsa qualità dell'aria     5
01/03/2005 8546  01/03/2005  20.00.00     2.8  scarsa qualità dell'aria     9
           8547  01/03/2005  21.00.00     2.2  scarsa qualità dell'aria     9
           8545  01/03/2005  19.00.00     2.1  scarsa qualità dell'aria     9
01/04/2004 518   01/04/2004  08.00.00     6.2  scarsa qualità dell'aria    14


C:\Users\PC467PH\AppData\Local\Temp\ipykernel_16724\1400446273.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  higher = df.groupby('Date').apply(lambda g: g.nlargest(3, 'CO(GT)'))


In [25]:
# Percentuale settimanale
df['Week'] = pd.to_datetime(df['Date'], dayfirst=True).dt.isocalendar().week
weekly = df.groupby('Week')['Label'].apply(lambda x: (x == "scarsa qualità dell'aria").mean()*100)
print(weekly)

Week
1     46.666667
2     51.219512
3     49.629630
4     50.704225
5     43.030303
6     52.409639
7     43.975904
8     43.636364
9     42.168675
10    34.615385
11    41.984733
12    43.425076
13    43.768997
14    45.070423
15    43.589744
16    39.215686
17    46.153846
18    45.962733
19    44.444444
20    40.789474
21    45.962733
22    43.750000
23    45.360825
24    44.848485
25    47.878788
26    45.783133
27    44.578313
28    42.168675
29    44.242424
30    41.818182
31    36.956522
32    48.351648
33    50.303030
34    45.312500
35    47.154472
36    38.383838
37    46.534653
38    46.153846
39    46.666667
40    45.098039
41    48.101266
42    45.714286
43    47.000000
44    47.204969
45    48.076923
46    42.567568
47    48.170732
48    51.533742
49    43.373494
50    51.204819
51    48.484848
52    49.397590
53    50.000000
Name: Label, dtype: float64


In [27]:
global_mean = (df['Label'] == "scarsa qualità dell'aria").mean()*100
print(global_mean)

45.49126922074537


In [28]:
# Random forest
df['Hour'] = pd.to_datetime(df['Time'], format='%H.%M.%S').dt.hour
X = df[['Hour', 'CO(GT)']]
y = (df['Label'] == "scarsa qualità dell'aria").astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.8221498371335505
